In [21]:
import pandas as pd
import geojson
import openrouteservice
import time
import os
import folium
import json

import os
from dotenv import load_dotenv
load_dotenv('.env')

True

In [22]:
# authentication OpenRouteService
secret = os.getenv("ORS_SECRET")
ors = openrouteservice.Client(key=secret)

In [36]:
# load CSV
poi = pd.read_csv('./dummy/poi.csv')

posters = pd.read_csv('./dummy/p.csv')



In [37]:
# extract coords
poi_coords = poi[["longitude", 'latitude']].values.tolist()
poster_coords= posters[["longitude",'latitude']].values.todict()

[[4.4158472, 51.2452363]]

In [27]:
# make geojson files for each pair of coords
# for item in poi_coords:
   
#     isochrone = ors.isochrones(locations=[item], range=[600])
#     with open('./dummy/test_geo.geojson', 'w') as f:
#         json.dump(isochrone, f, ensure_ascii=False,indent=4)

In [28]:
# load geojson file into the program and check that it is a valid object
with open('./dummy/test_geo.geojson', 'r') as f:
    gj = geojson.load(f)

gj.is_valid

True

In [29]:
# reverse coords for use in folium
for item in poi_coords:
    item.reverse()

In [6]:
poi_coords

[[51.2452363, 4.4158472],
 [51.2186918, 4.4055072],
 [50.8141563, 4.4257236],
 [50.8598131, 5.5003105],
 [51.2085548, 3.2240359]]

In [34]:
map = folium.Map(location=(poi_coords[0]), tiles="cartodbpositron", min_zoom=11)

folium.Marker(poi_coords[0]).add_to(map)

folium.features.GeoJson(gj).add_to(map)

map